<a href="https://colab.research.google.com/github/mehdihatami1998/DynamicsOfStructures/blob/main/L14_MatrixIteration_RayleighRitz_SubspaceIteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.linalg import eigh

from IPython.display import Math

def pmat(mat, fmt='%.3g', col=True, bracket='b'):
    env = bracket + 'matrix'
    if len(mat.shape)==1:    # mat is a 1D array
        mat = mat[:,None] if col else mat[None,:]
    if len(mat.shape) !=2:
        raise ValueError('Only 1D e 2D arrays are supported')
    return r'\begin{%s}%s\end{%s}'%(env, 
      r'\\'.join('&'.join(fmt%elt for elt in row) for row in mat), env)

def math(*l):
    display(Math('{}'.join(el for el in l)))

# Matrix Iteration, Rayleigh–Ritz, Subspace Iteration

## Problem Statement

We are studying a shear type frame with $n_s$ storeys, where all floor masses are the same, $m_i=m,\ i=1,\ldots,n_s$, and also the lateral storey stiffnesses are the same¸ $k_i=k,\ i=1,\ldots,n_s$.

### Number of stories

In [ ]:
ns = 12

### Structural Matrices

The mass matrix is $\mathbf M = m\,\mathbf I$, the stiffness matrix is tridiagonal, with $k_{i,i}=k_i+k_{i+1}=2k$ and $k_{i,i+1}=k_{i+1,1}=-k_i=-k$.

In [ ]:
ones = np.ones(ns)
M = np.diag(ones)
K = np.diag(ones*2)
K[-1,-1] = 1 # Because there is no other storey stiffness above the top...
K = K - np.diag(ones[1:], -1) - np.diag(ones[1:], +1)

Possibly print the matrices… (toggle `False` with `True` — capitalization is important)

In [ ]:
print_matrices = False
if print_matrices:
    math(r'\boldsymbol K = k\,', pmat(K))
    math(r'\boldsymbol M = m\,', pmat(M))

### Our reference solution
Compute the eigenvalues and eigenvectors using an algebra library, print the first 6 eigenvalues normalized wrt $\omega_0^2=k/m$.

In [ ]:
evals, evecs = eigh(K, M)
print(evals[:6])

## Graphical Output
### Function definition
Define a function to help us showing our results.

Important aspects of plotting these eigenvectors:
1. the independent variable (the storey level) must be on the $y$ axis, the dependent variable (the storey lateral displacement) must be on the $x$ axis,
1. we add to each eigenvector a zero term, to correctly plot the zero ground displacement.

In [ ]:
def eigenplot(ns, evecs, n_evecs=None, norm=False, fig_ax=None, title=None):
    if fig_ax is None:
        fig, ax = plt.subplots(figsize=(4,6), constrained_layout=True)
    else:
        fig, ax = fig_ax   
    if n_evecs is None: n_evecs = ns 
    x = np.arange(ns+1)
    y = evecs/(np.abs(evecs).max(axis=0) if norm else 1)/np.sign(evecs[-1]) 
    y = np.vstack((np.zeros(y.shape[1]), y))
    for i, evec in enumerate((y.T)[:n_evecs], 1): 
        ax.plot(evec, x, label='$\\psi_{%d}$'%i)
    ax.legend() 
    ax.grid(visible=1, axis='y')
    ax.set_yticks(range(ns+1))
    #x.yaxis.set_major_locator(plt.MultipleLocator(1))
    if title : ax.set_title(title)

### Check the function
Using it to plot the first 6 eigenvectors

In [ ]:
eigenplot(ns, evecs, n_evecs=6, norm=True)

## Matrix Iteration

Compute 3 eigenvectors, using matrix iteration with sweeps.

Initialize the dynamic matrix $\mathbf D_0 = \mathbf K^{-1}\mathbf M$, and the sweeping matrix, $\mathbf S = \mathbf I$, then compute a mode using the modified dynamic matrix, after convergence update the sweeping matrix and compute another mode, until 3 modes have been computed.

In [ ]:
D0 = np.linalg.inv(K) @ M
S = np.diag(ones)
sevals, sevecs = [], []
for i in range(3):
    D, x, w2old = D0 @ S, ones, 0.0
    for _ in range(25):
        x_hat = D @ x
        w2 = (x_hat @ M @ x) / (x_hat @ M @ x_hat)
        x = x_hat*w2
        if abs(w2-w2old)/w2 < 1E-8 : break
        w2old = w2
    else:
        print('No convergence in 25 iterations') ; 1/0
    sevals.append(w2), sevecs.append(x)
    modal_m = x.T@M@x
    S = S - np.outer(x,x)@M/modal_m
sevals, sevecs = np.array(sevals), np.array(sevecs).T

Compare the exact eigenvalues (first row) and those computed using iteration with sweeps.

In [ ]:
print(evals[:3])
print(sevals)


Now plot, side by side the eigenvectors computed by iteration with sweeps and the algebraic library — they are the same.

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(8,8), constrained_layout=True)
eigenplot(ns, sevecs, n_evecs=3, norm=1, fig_ax=(fig, axes[0]), title='Matrix Iteration')
eigenplot(ns, evecs,  n_evecs=3, norm=1, fig_ax=(fig, axes[1]), title='"Exact" Eigenvectors')

## Ritz-Rayleigh

Use $M=8$ Ritz base vectors, here they are simply random vectors, you can do much better when you choose a more appropriate starting base (e.g., "Derived Ritz Vectors" in Clough Penzien).
With our inefficient base, we compute the reduced matrices, the Ritz eigen* and finally the eigenvectors in structural coordinates.

In [ ]:
M_base = 8
ns = 12
np.random.seed(20220512)
phi = D0@(np.random.random((ns, M_base))-0.5)
# uncomment the following two lines and re-execute this cell
# to see what a wisely chosen base can do
# phi = np.sin(np.outer((1+np.arange(ns))/ns,
#             np.pi*(np.arange(M_base)*2+1)/2 ))
k, m = phi.T@K@phi, phi.T@M@phi
zevals, zevecs = eigh(k, m)
psi = phi@zevecs

Print the percent errors for all the $M$ eigenvalues

In [ ]:
print('(Mode no., error%)', 
      *enumerate(100*(zevals-evals[:M_base])/evals[:M_base], 1), sep='\n')

and eventually plot, side by side, a few of the RR eigenvectors and the real ones.

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(8,8), constrained_layout=True)
eigenplot(ns, psi,   n_evecs=6, norm=0, fig_ax=(fig, axes[0]), title='Rayleigh-Ritz')
eigenplot(ns, evecs, n_evecs=6, norm=0, fig_ax=(fig, axes[1]), title='"Exact" Eigenvectors')

## Subspace Iteration no.1

4 Ritz vectors

In [ ]:
np.random.seed(20220512)
psi = np.random.random((ns, 4))
for i in range(2):
    phi = D0@psi
    k, m = phi.T@K@phi, phi.T@M@phi
    zevals, zevecs = eigh(k, m)
    psi = phi@zevecs
print('Ex', evals[:4])
print('SI', zevals[:4])

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(8,8), constrained_layout=True)
eigenplot(ns, psi,   n_evecs=4, norm=1, fig_ax=(fig, axes[0]), title='2 Subspace Iterations, M=4')
eigenplot(ns, evecs, n_evecs=4, norm=1, fig_ax=(fig, axes[1]), title='"Exact" Eigenvectors')

## Subspace Iteration no. 2

8 Ritz vectors

In [ ]:
np.random.seed(20220512)
psi = np.random.random((ns, 8))
for i in range(2):
    phi = D0@psi
    k, m = phi.T@K@phi, phi.T@M@phi
    zevals, zevecs = eigh(k, m)
    psi = phi@zevecs
print('Exact   ', evals[:4])
print('SubSpace', zevals[:4])

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(8,8), constrained_layout=True)
eigenplot(ns, psi,   n_evecs=4, norm=1, fig_ax=(fig, axes[0]), title='2 Subspace Iterations, M=8')
eigenplot(ns, evecs, n_evecs=4, norm=1, fig_ax=(fig, axes[1]), title='"Exact" Eigenvectors')

### A Larger Problem…



In [ ]:
########
ns =  50
########
big_ones = np.ones(ns)
Mbig = np.diag(big_ones)
Kbig = Mbig*2
Kbig[-1,-1] = 1
Kbig = (Kbig - np.diag(big_ones[1:], -1) - np.diag(big_ones[1:], +1))*1000
evals, evecs = eigh(Kbig, Mbig)
D0 = np.linalg.inv(Kbig)@Mbig

In [ ]:
np.random.seed(20220512)
psi = np.random.random((ns, 4))
for i in range(3):
    phi = D0@psi
    k, m = phi.T@Kbig@phi, phi.T@Mbig@phi
    zevals, zevecs = eigh(k, m)
    psi = phi@zevecs
print('Ex', evals[:4])
print('SI', zevals[:4])

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(8,8), constrained_layout=True)
eigenplot(ns, psi,   n_evecs=4, norm=0, fig_ax=(fig, axes[0]), title='2 Subspace Iterations, M=4')
eigenplot(ns, evecs, n_evecs=4, norm=0, fig_ax=(fig, axes[1]), title='"Exact" Eigenvectors')

In [ ]:
np.random.seed(20220512)
psi = np.random.random((ns, 6))
for i in range(4):
    phi = D0@psi
    k, m = phi.T@Kbig@phi, phi.T@Mbig@phi
    zevals, zevecs = eigh(k, m)
    psi = phi@zevecs
print('                    Eigenvalues 1 to 4')
print('             Exact', evals[:4])
print('Subspace Iteration', zevals[:4])

In [ ]:
fig, axes = plt.subplots(1,2,figsize=(8,8), constrained_layout=True)
eigenplot(ns, psi,   n_evecs=4, norm=1, fig_ax=(fig, axes[0]), title='2 Subspace Iterations, M=6')
eigenplot(ns, evecs, n_evecs=4, norm=1, fig_ax=(fig, axes[1]), title='"Exact" Eigenvectors')